In [ ]:
# ライブラリ

In [ ]:
import os
import sys
import glob
from IPython.display import display
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
#import japanize_matplotlib
import seaborn as sns 
import sweetviz as sv
import yaml

sys.path.append(os.path.abspath('..'))
from configs.config import *

In [ ]:
import importlib
import configs.config
importlib.reload(configs.config)
from configs.config import *

In [ ]:
pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows", 500)

# データ読み込み

In [ ]:
df_train = pd.read_csv(os.path.join(DIR_INPUT, FILE_NAME_TRAIN))
df_test = pd.read_csv(os.path.join(DIR_INPUT, FILE_NAME_TEST))
df_sample_submission = pd.read_csv(os.path.join(DIR_INPUT, FILE_SAMPLE_SUBMISSION))
df_udemy_activity = pd.read_csv(os.path.join(DIR_INPUT, FILE_NAME_UDEMY_ACTIVITY))
df_career = pd.read_csv(os.path.join(DIR_INPUT, FILE_NAME_CAREER))
df_dx = pd.read_csv(os.path.join(DIR_INPUT, FILE_NAME_DX))
df_hr = pd.read_csv(os.path.join(DIR_INPUT, FILE_NAME_HR))
df_overtime_work_by_month = pd.read_csv(os.path.join(DIR_INPUT, FILE_NAME_ORVER_TIME))
df_position_history = pd.read_csv(os.path.join(DIR_INPUT, FILE_NAME_POSITION_HISTORY))

In [ ]:
print(f"train shape: {df_train.shape}")
print(f"test shape: {df_test.shape}")  
print(f"sample submission shape: {df_sample_submission.shape}")
print(f"udemy activity shape: {df_udemy_activity.shape}")
print(f"career shape: {df_career.shape}")
print(f"dx shape: {df_dx.shape}")
print(f"hr shape: {df_hr.shape}")
print(f"overtime work by month shape: {df_overtime_work_by_month.shape}")
print(f"position history shape: {df_position_history.shape}")

# 型、項目数、欠損値率

In [ ]:
def calc_describe(df):
    dtypes = []
    val_counts_station = []
    isnull_station = []
    isnull_station_ratio = 100 * df.isnull().sum() / len(df)
    for col in df.columns:
        dtypes.append(str(df[col].dtype))
        val_counts_station.append(len(df[col].value_counts()))
        isnull_station.append(isnull_station_ratio[col])
    inds = ["型", "val_counts", "NaN率"]
    df_eda = pd.DataFrame([dtypes, val_counts_station, isnull_station], columns=df.columns, index=inds).T
    # df_eda.query("val_counts > 1")
    return df_eda

In [ ]:
# 各データフレームのEDAを実行
for df, name in zip([df_train, df_test, df_sample_submission, df_udemy_activity, df_career, df_dx, df_hr, df_overtime_work_by_month, df_position_history],
             ["df_train", "df_test", "df_sample_submission", "df_udemy_activity", "df_career", "df_dx", "df_hr", "df_overtime_work_by_month", "df_position_history"]):
    print(f"{name}")
    df_eda = calc_describe(df)
    display(df_eda)

# 基本統計量

In [ ]:
for df, name in zip([df_train, df_test, df_sample_submission, df_udemy_activity, df_career, df_dx, df_hr, df_overtime_work_by_month, df_position_history],
             ["df_train", "df_test", "df_sample_submission", "df_udemy_activity", "df_career", "df_dx", "df_hr", "df_overtime_work_by_month", "df_position_history"]):
    print(f"{name}")
    display(df.head())

In [ ]:
for df, name in zip([df_train, df_test, df_sample_submission, df_udemy_activity, df_career, df_dx, df_hr, df_overtime_work_by_month, df_position_history],
             ["df_train", "df_test", "df_sample_submission", "df_udemy_activity", "df_career", "df_dx", "df_hr", "df_overtime_work_by_month", "df_position_history"]):
    print(f"{name}")
    display(df.describe())

# train

In [ ]:
df_train

In [ ]:
df_train['社員番号'].nunique()

In [ ]:
df_train['category'].value_counts()

In [ ]:
df_train.groupby('category')['target'].value_counts().reset_index()

# test

In [ ]:
df_test.head()

In [ ]:
df_test['category'].value_counts()

In [ ]:
df_test['社員番号'].nunique()

# udemy_activity

In [ ]:
df_udemy_activity

In [ ]:
df_udemy_activity[(df_udemy_activity['社員番号']=='zxY0Eflwm1tYj1Wt6vo_1g==')]

In [ ]:
df_udemy_target = pd.merge(
    df_train[df_train['target']==1], df_udemy_activity, how='left', on=['社員番号']
)

In [ ]:
df_udemy_target['社員番号'].unique()

In [ ]:
# 時系列の受講数を集計
df_udemy_target['開始年'] = pd.to_datetime(df_udemy_target['開始日']).dt.to_period('Y')
df_udemy_target['開始年月'] = pd.to_datetime(df_udemy_target['開始日']).dt.to_period('M')
df_udemy_target['開始年'].value_counts().sort_index().plot(kind='bar', figsize=(12, 6))
# df_udemy_train['開始年月'].value_counts().sort_index().plot(kind='bar', figsize=(12, 6))

In [ ]:
# 時系列の受講数を集計
df_udemy_activity['開始年'] = pd.to_datetime(df_udemy_activity['開始日']).dt.to_period('Y')
df_udemy_activity['開始年月'] = pd.to_datetime(df_udemy_activity['開始日']).dt.to_period('M')
df_udemy_activity['開始年'].value_counts().sort_index().plot(kind='bar', figsize=(12, 6))
# df_udemy_activity[df_udemy_activity['社員番号']=='zxY0Eflwm1tYj1Wt6vo_1g==']['開始年'].value_counts().sort_index().plot(kind='bar', figsize=(12, 6))

In [ ]:
i = 10
df_udemy_target[df_udemy_target['社員番号']==df_udemy_target['社員番号'].unique()[i]]['開始年'].value_counts().sort_index().plot(kind='bar', figsize=(12, 6))

In [ ]:
df_train[df_train['target']==0]['社員番号'].unique()[11]

In [ ]:
df_udemy_activity[df_udemy_activity['社員番号']==df_train[df_train['target']==0]['社員番号'].unique()[21]]['開始年'].value_counts().sort_index().plot(kind='bar', figsize=(12, 6))

In [ ]:
df_udemy_train

In [ ]:
df_udemy_activity.dtypes

In [ ]:
df_udemy_activity['コースタイトル'].value_counts().reset_index()

In [ ]:
df_udemy_activity['レクチャーもしくはクイズ'].value_counts().reset_index()

In [ ]:
df_udemy_activity['レクチャー/クイズの題名'].value_counts().reset_index()

In [ ]:
df_udemy_activity['コースカテゴリー'].value_counts().reset_index().head(10)

In [ ]:
df_udemy_activity['コースタイトル'].nunique()

In [ ]:
df_udemy_activity['コースID'].nunique()

# career

In [ ]:
df_career.head()

# dx

In [ ]:
df_dx

In [ ]:
df_dx['研修カテゴリ'].value_counts()

In [ ]:
df_dx['研修名'].value_counts()

# hr

In [ ]:
df_hr

In [ ]:
df_hr['カテゴリ'].value_counts()

In [ ]:
df_hr['研修名'].value_counts()

In [ ]:
df_hr[df_hr['実施日']=='2024/5/17,2024/7/83']

In [ ]:
df_hr[df_hr['社員番号']=='zxmtr2h4ypvsNq02K9AMJg==']

In [ ]:
df_hr[df_hr['カテゴリ']=='キャリア面談'].head(10)

In [ ]:
df_hr['実施日'].value_counts()

# overtime_work_by_month

In [ ]:
df_overtime_work_by_month

In [ ]:
# 欠損値の確認
print("Missing values:")
print(df_overtime_work_by_month.isnull().sum())

# 基本統計量の確認
print("\nBasic statistics:")
display(df_overtime_work_by_month.describe())

# hours列の分布を可視化
plt.figure(figsize=(10, 6))
sns.histplot(df_overtime_work_by_month['hours'], bins=30, kde=True)
plt.title('Distribution of Overtime Hours')
plt.xlabel('Hours')
plt.ylabel('Frequency')
plt.show()

# 社員番号ごとの平均残業時間
avg_hours_per_employee = df_overtime_work_by_month.groupby('社員番号')['hours'].mean().reset_index()
avg_hours_per_employee.columns = ['社員番号', '平均残業時間']
print("\nTop 5 employees with highest average overtime hours:")
display(avg_hours_per_employee.sort_values(by='平均残業時間', ascending=False).head())

In [ ]:
df_overtime_work_by_month.groupby('社員番号')['hours'].count().value_counts().reset_index().sort_values(by='hours', ascending=False)

In [ ]:
df_overtime_work_by_month['date'].value_counts().reset_index().sort_values(by='date', ascending=True)

In [ ]:
df_overtime_work_by_month[df_overtime_work_by_month['社員番号']=='hT9YU8Cc2gXCE8eoAA4xOA==']

In [ ]:
df_overtime_work_by_month[df_overtime_work_by_month['date']=='2024-12-01']

In [ ]:
df_overtime_work_by_month['社員番号'].nunique()

# position_history

In [ ]:
df_position_history

In [ ]:
df_position_history['社員番号'].nunique()

In [ ]:
df_train_ph = pd.merge(df_train, df_position_history, on='社員番号', how='left')

In [ ]:
df_position_history['year'].hist(bins=20)
df_train_ph[df_train_ph['target']==1]['year'].hist(bins=20)

In [ ]:
df_position_history.drop_duplicates(subset=['社員番号'], keep='last')['year'].hist(bins=20)

In [ ]:
df_position_history[df_position_history['year']==22]

In [ ]:
df_position_history[df_position_history['year']==24]

In [ ]:
df_position_history[df_position_history['社員番号']=='-4jh26kLzkU8JFQwdeQU9w==']

In [ ]:
df_position_history.dtypes

In [ ]:
df_position_history['勤務区分'].value_counts()

In [ ]:
df_position_history['役職'].value_counts()

# 前処理済データの読み込み

In [ ]:
df_prep_station = pd.read_pickle(os.path.join(DIR_INTERIM, "df_prep_station.pkl"))
df_prep_status = pd.read_pickle(os.path.join(DIR_INTERIM, "df_prep_status.pkl"))
df_prep_trip = pd.read_pickle(os.path.join(DIR_INTERIM, "df_prep_trip.pkl"))
df_prep_weather = pd.read_pickle(os.path.join(DIR_INTERIM, "df_prep_weather.pkl"))